自动化去水印(智能抠图!)

Step 1:
通过BirefNet得到商品主体蒙版

step2:
将主体部分inpaint掉，防止干扰 (存疑，或许不需要，也有可能会导致inpaint效果不佳)

Step 3:
OCR识别每组文字，得到每组文字的边界框

Step 4:
每组文字的边界框和中心点作为segment anything的输入，得到每组文字的蒙版

Step 5:
将文字蒙版合并，inpaint，得到无文字的干净背景

Step 6:
将商品主体蒙版和无文字的干净背景融合，得到最终结果




In [303]:
# import torch
# import gc
# import psutil
# import os

# def clear_all_memory():
#     """清理内存函数"""
#     # 清理 CUDA 缓存
#     if torch.cuda.is_available():
#         torch.cuda.empty_cache()
#         print(f"GPU内存已清空")
#         print(f"当前GPU内存占用：{torch.cuda.memory_allocated()/1024**2:.2f} MB")
    
#     # 清理 CPU 缓存
#     gc.collect()
    
#     # 获取进程信息
#     process = psutil.Process(os.getpid())
#     print(f"CPU内存占用：{process.memory_info().rss/1024/1024:.2f} MB")
#     print("内存清理完成")

# clear_all_memory()

In [364]:
test_img_dir = '/root/rm_water_mark/test_imgs'
img_name = '11'
img_path = f'/root/rm_water_mark/test_imgs/{img_name}.jpg'
product_mask_path = f'/root/rm_water_mark/test_imgs/{img_name}_mask.jpg'
pic_no_prod_path = f'/root/rm_water_mark/test_imgs/{img_name}_no_prod.jpg'
water_mark_mask_path = f'/root/rm_water_mark/test_imgs/{img_name}_water_mark_mask.jpg'
pic_clean_bg_path = f'/root/rm_water_mark/test_imgs/{img_name}_clean_bg.jpg'
final_img_path = f'/root/rm_water_mark/test_imgs/{img_name}_rm.jpg'

In [365]:
import time

start_time = time.time()

In [366]:
import cv2

# 读取原图
# original_image = Image.open(img_path)
original_image = cv2.imread(img_path)

通过rmbg得到原图+主体蒙版

In [367]:
import cv2
import matplotlib.pyplot as plt
import sys
from pathlib import Path
import numpy as np
import torch
from PIL import Image
from torchvision import transforms
from transformers import AutoModelForImageSegmentation

MODEL_DIR = '/root/autodl-tmp/RMBG-2.0'

input_image_size = (1024, 1024)
transform_image = transforms.Compose([
    transforms.Resize(input_image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# image = cv2.imread(image_path)
# mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

In [368]:
def predict(model, input_images, original_size):
    with torch.no_grad():
        preds = model(input_images)[-1].sigmoid().cpu()
    pred = preds[0].squeeze()
    pred_pil = transforms.ToPILImage()(pred)
    mask = pred_pil.resize(original_size)
    return mask

def run_inference(image: Image):
    """
    result_image = run_inference(image)
    """
    # image
    original_size = image.size

    # load model
    model = AutoModelForImageSegmentation.from_pretrained(MODEL_DIR, trust_remote_code=True)
    torch.set_float32_matmul_precision(['high', 'highest'][0])
    model.to('cuda')
    model.eval()

    # process img
    input_images = transform_image(image).unsqueeze(0).to('cuda')
    # predict
    mask = predict(model, input_images, original_size)
    # # post process
    # image.putalpha(mask)
    # # 创建白色背景
    # white_background = Image.new("RGBA", original_size, (255, 255, 255, 255))
    # # 将透明图像与白底合成
    # composite = Image.alpha_composite(white_background, image)
    # final_img = composite.convert("RGB")

    return mask


In [369]:
product_mask = run_inference(Image.fromarray(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)))
product_mask.save(product_mask_path)

In [370]:
# 将product_mask转换为cv2格式
product_mask = np.array(product_mask)

使用Inpaint anything 把商品主体(蒙版部分)消除

In [371]:
import sys
sys.path.insert(0, '/root/Inpaint-Anything-main')
sys.path.insert(0, '/root/Inpaint-Anything-main/..')

from lama_inpaint import load_inpaint_model, inpaint_img_with_model

In [372]:
from pathlib import Path

INPAINT_ANYTHING_DIR = Path(r'/root/Inpaint-Anything-main')
LAMA_CONFIG = INPAINT_ANYTHING_DIR.joinpath('lama', 'configs', 'prediction', 'default.yaml')
LAMA_CKPT = '/root/autodl-tmp/pretrained_models/big-lama'

lama_predictor, lama_predict_config = load_inpaint_model(LAMA_CONFIG, LAMA_CKPT)

cv_kernel = np.ones((5, 5), dtype=np.uint8)
#先对蒙版进行膨胀操作
mask = cv2.dilate(product_mask, cv_kernel, iterations=3)

def inpaint_img(image_array, mask):
    img_inpainted = inpaint_img_with_model(img=image_array,
                                           mask=mask,
                                           model=lama_predictor,
                                           predict_config=lama_predict_config)
    return img_inpainted

image_array = np.array(original_image)
mask_array = np.array(mask)
img_inpainted = inpaint_img(image_array, mask_array)

In [373]:
cv2.imwrite(pic_no_prod_path, img_inpainted)

True

ocr识别文字组

没有用原先的pytesseract，使用了 paddle ocr

In [374]:
from paddleocr import PaddleOCR
ocr = PaddleOCR(
    use_angle_cls=True, 
    lang='ch',
    det_model_dir='/root/autodl-tmp/paddleocr/det/ch_ppocr_server_v2.0_det_infer/',
    rec_model_dir='/root/autodl-tmp/paddleocr/rec/ch_ppocr_server_v2.0_rec_infer/',
    cls_model_dir='/root/autodl-tmp/paddleocr/cls/ch_ppocr_mobile_v2.0_cls_infer/',
    ) # need to run only once to download and load model into memory

# img_path = '/root/rm_water_mark/imgs/image2_inpainted.jpg'
# 注意 ocr.ocr()如果传入的是ndarray,结果需要去result[0]获取
result = ocr.ocr(img_inpainted, cls=True)    # image path / ndarray
if len(result) == 0:
    print('No text detected!')
    pass

for idx in range(len(result)):
    print(idx)
    res = result[idx]
    for line in res:
        print(line)


[2024/12/26 19:28:57] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/autodl-tmp/paddleocr/det/ch_ppocr_server_v2.0_det_infer/', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/autodl-tmp/paddleocr/rec/ch_ppocr_server_v2.0_rec_infer/', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, 

In [375]:
# boxes = [line[0] for line in result]
boxes = [line[0] for line in result[0]]
# 计算中心点
def get_center(box):
    x = (box[0][0] + box[2][0]) / 2
    y = (box[0][1] + box[2][1]) / 2
    return [x, y]

# 增加中心点作为box的第五个点
def add_center(boxes):
    for box in boxes:
        center = get_center(box)
        box.append(center)
    return boxes

# boxes = add_center(boxes)


使用segment anything 得到文字蒙版

In [376]:
import numpy as np
import torch
from segment_anything import sam_model_registry, SamPredictor
import cv2


In [377]:
SAM_MODEL_TYPE = 'vit_h'
SAM_CKPT = '/root/autodl-tmp/pretrained_models/sam_vit_h_4b8939.pth'
DEVICE = 'cuda'

def initialize_sam(checkpoint_path, model_type="vit_h", device="cuda"):
    """ 初始化 SAM 模型 """
    sam = sam_model_registry[model_type](checkpoint=checkpoint_path)
    sam.to(device)
    predictor = SamPredictor(sam)
    return predictor

def get_mask_from_points(predictor, image, points):
    """
    从点集获取mask
    Args:
        predictor: SAM 预测器
        image:输入图像(nparray, RGB)
        points: 点的坐标列表 [x1, y1], [x2, y2], ...
    Returns:
        masks: 生成的蒙版
        scores: 蒙版的置信度
    """
    predictor.set_image(image)
    input_points = np.array(points)
    point_labels = np.ones(len(points))

    masks, scores, logits = predictor.predict(
        point_coords = input_points,
        point_labels = point_labels,
        multimask_output=True,
    )

    return masks, scores

def process_mask(mask, dilate_kernel_size=5, num_iterations=2):
    """ 膨胀蒙版 """
    # 将bool转为uint8
    mask_uint8 = mask.astype(np.uint8) * 255

    # 创建膨胀核
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (dilate_kernel_size, dilate_kernel_size))
    # 膨胀
    dilated_mask = cv2.dilate(mask_uint8, kernel, iterations=num_iterations)
    blurred_mask = cv2.GaussianBlur(dilated_mask, (5, 5), 0)
    # 转回布尔类型
    final_mask = blurred_mask > 127

    return final_mask

In [378]:
predictor = initialize_sam(SAM_CKPT, SAM_MODEL_TYPE, DEVICE)

samed_masks = []
for points in boxes:
    masks, scores = get_mask_from_points(predictor, img_inpainted, points)
    
    best_mask_index = np.argmax(scores)
    best_mask = masks[best_mask_index]

    processed_mask = process_mask(
        best_mask,
        dilate_kernel_size=5,
        num_iterations=2
    )
    samed_masks.append(processed_mask)



读取模型文件/root/autodl-tmp/pretrained_models/sam_vit_h_4b8939.pth


In [379]:
# 将boxes转换为box_masks
def get_box_mask(box, image_shape):
    mask = np.zeros(image_shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, [np.array(box, dtype=np.int32)], 1)
    # 膨胀处理
    cv_kernel = np.ones((5, 5), dtype=np.uint8)
    mask = cv2.dilate(mask, cv_kernel, iterations=3)
    return mask

def get_boxes_mask(boxes, image_shape):
    masks = []
    for box in boxes:
        mask = get_box_mask(box, image_shape)
        masks.append(mask)
    return masks

boxes_mask = get_boxes_mask(boxes, img_inpainted.shape)

In [380]:
for box_mask in boxes_mask:
    print(np.sum(box_mask))
print('------------------')
for samed_mask in samed_masks:
    print(np.sum(samed_mask))

6560
38726
------------------
697195
696049


In [381]:
from typing import List
def get_final_masks(boxes_mask, 
                    samed_masks,
                    area_ratio_threshold: float = 2.0,
                    iou_threshold: float = 0.3,      # 用于判断两个SAM框是否相似
                    distance_threshold: float = 50    # 用于判断两个框是否足够接近
                    ) -> List[List[List[float]]]:
    """
    根据原始文本框和SAM处理后的框来决定最终使用的框
    
    Args:
        boxes_mask: 由文字box掩码组成的列表
        samed_masks: SAM处理后的掩码列表
        area_ratio_threshold: SAM框与原始框面积比的阈值，默认2.0
        iou_threshold: 判断两个SAM框相似度的IOU阈值
        distance_threshold: 判断两个框是否接近的距离阈值（像素）
        
    Returns:
        最终确定的框列表
    """
    final_masks = []
    n = len(boxes_mask)
    
    # 首先检查相邻的SAM框是否构成文本框
    text_block_groups = []  # 存储可能属于同一个文本框的mask组
    processed = set()       # 记录已经处理过的mask索引
    
    def calculate_box_distance(mask1, mask2):
        """计算两个掩码的中心点距离"""
        y1, x1 = np.mean(np.where(mask1), axis=1)
        y2, x2 = np.mean(np.where(mask2), axis=1)
        return np.sqrt((x2-x1)**2 + (y2-y1)**2)
    
    def calculate_iou(mask1, mask2):
        """计算两个掩码的IOU"""
        intersection = np.sum(mask1 & mask2)
        union = np.sum(mask1 | mask2)
        return intersection / union if union > 0 else 0
    
    # 寻找可能属于同一文本框的mask组
    for i in range(n):
        if i in processed:
            continue
            
        current_group = [i]
        for j in range(i+1, n):
            if j in processed:
                continue
                
            # 计算两个SAM掩码的相似度和距离
            iou = calculate_iou(samed_masks[i], samed_masks[j])
            distance = calculate_box_distance(samed_masks[i], samed_masks[j])
            
            # 如果两个掩码足够相似且距离适中，认为它们可能属于同一个文本框
            if iou > iou_threshold and distance < distance_threshold:
                current_group.append(j)
                processed.add(j)
                
        if len(current_group) > 1:  # 如果找到多个相关的掩码
            text_block_groups.append(current_group)
        processed.add(i)


    ungroup_index = [] # 用于记录不属于任何文本块组的mask索引
    # 根据text_block_groups制作grouping_masks 然后使用面积比逻辑
    for group in text_block_groups:
        grouping_mask = []
        idx_list = []
        for idx in group:
            grouping_mask.append(boxes_mask[idx])
            idx_list.append(idx)
        group_area = np.sum(grouping_mask)
        samed_area = np.sum(samed_masks[idx_list[0]])
        area_ratio = samed_area / group_area if group_area > 0 else float('inf')

        if area_ratio > area_ratio_threshold:
            ungroup_index.extend(idx_list)

    
    # 处理每个mask
    for i in range(n):
        if i in ungroup_index:  
            final_masks.append(boxes_mask[i])
            continue

        # 检查当前mask是否属于某个文本块组
        is_in_text_block = False
        for group in text_block_groups:
            if i in group:
                is_in_text_block = True
                # 如果是组内第一个mask，合并该组所有mask
                if i == group[0]:
                    merged_mask = np.zeros_like(samed_masks[i], dtype=bool)
                    for idx in group:
                        merged_mask |= samed_masks[idx]
                    final_masks.append(merged_mask)
                break
        
        # 如果不属于任何文本块组，使用原有的面积比逻辑
        if not is_in_text_block:
            orig_area = np.sum(boxes_mask[i])
            samed_area = np.sum(samed_masks[i])
            area_ratio = samed_area / orig_area if orig_area > 0 else float('inf')
            
            if area_ratio > area_ratio_threshold:
                final_masks.append(boxes_mask[i])
            else:
                final_masks.append(samed_masks[i])
    
    return final_masks


In [382]:
# from typing import List

# def get_final_masks(boxes_mask, 
#                     samed_masks,
#                     area_ratio_threshold: float = 2.0,
#                     iou_threshold: float = 0.3,      # 用于判断两个SAM框是否相似
#                     distance_threshold: float = 50    # 用于判断两个框是否足够接近
#                     ) -> List[List[List[float]]]:
#     """
#     根据原始文本框和SAM处理后的框来决定最终使用的框
    
#     Args:
#         boxes_mask: 由文字box掩码组成的列表
#         samed_masks: SAM处理后的掩码列表
#         area_ratio_threshold: SAM框与原始框面积比的阈值，默认2.0
#         iou_threshold: 判断两个SAM框相似度的IOU阈值
#         distance_threshold: 判断两个框是否接近的距离阈值（像素）
        
#     Returns:
#         最终确定的框列表
#     """
#     final_masks = []
#     n = len(boxes_mask)
    
#     # 首先检查相邻的SAM框是否构成文本框
#     text_block_groups = []  # 存储可能属于同一个文本框的mask组
#     processed = set()       # 记录已经处理过的mask索引
    
#     def calculate_box_distance(mask1, mask2):
#         """计算两个掩码的中心点距离"""
#         y1, x1 = np.mean(np.where(mask1), axis=1)
#         y2, x2 = np.mean(np.where(mask2), axis=1)
#         return np.sqrt((x2-x1)**2 + (y2-y1)**2)
    
#     def calculate_iou(mask1, mask2):
#         """计算两个掩码的IOU"""
#         intersection = np.sum(mask1 & mask2)
#         union = np.sum(mask1 | mask2)
#         return intersection / union if union > 0 else 0
    
#     # 寻找可能属于同一文本框的mask组
#     for i in range(n):
#         if i in processed:
#             continue
            
#         current_group = [i]
#         for j in range(i+1, n):
#             if j in processed:
#                 continue
                
#             # 计算两个SAM掩码的相似度和距离
#             iou = calculate_iou(samed_masks[i], samed_masks[j])
#             distance = calculate_box_distance(samed_masks[i], samed_masks[j])
            
#             # 如果两个掩码足够相似且距离适中，认为它们可能属于同一个文本框
#             if iou > iou_threshold and distance < distance_threshold:
#                 current_group.append(j)
#                 processed.add(j)
                
#         if len(current_group) > 1:  # 如果找到多个相关的掩码
#             text_block_groups.append(current_group)
#         processed.add(i)
    
#     # 处理每个mask
#     for i in range(n):
#         # 检查当前mask是否属于某个文本块组
#         is_in_text_block = False
#         for group in text_block_groups:
#             if i in group:
#                 is_in_text_block = True
#                 # 如果是组内第一个mask，合并该组所有mask
#                 if i == group[0]:
#                     merged_mask = np.zeros_like(samed_masks[i], dtype=bool)
#                     for idx in group:
#                         merged_mask |= samed_masks[idx]
#                     final_masks.append(merged_mask)
#                 break
        
#         # 如果不属于任何文本块组，使用原有的面积比逻辑
#         if not is_in_text_block:
#             orig_area = np.sum(boxes_mask[i])
#             samed_area = np.sum(samed_masks[i])
#             area_ratio = samed_area / orig_area if orig_area > 0 else float('inf')
            
#             if area_ratio > area_ratio_threshold:
#                 final_masks.append(boxes_mask[i])
#             else:
#                 final_masks.append(samed_masks[i])
    
#     return final_masks



In [383]:
# from typing import List

# def get_final_masks(boxes_mask, 
#                    samed_masks,
#                    area_ratio_threshold: float = 2.0) -> List[List[List[float]]]:
#     """
#     根据原始文本框和SAM处理后的框来决定最终使用的框
    
#     Args:
#         boxes: 由文字box组成的列表，每个文字box内包含由4个坐标点组成的列表，每个坐标点为[x, y]
#         samed_boxes: 同上，为SAM处理后的框
#         area_ratio_threshold: SAM框与原始框面积比的阈值，默认2.0
        
#     Returns:
#         最终确定的框列表
#     """
#     final_masks = []
    
#     for box_mask, samed_mask in zip(boxes_mask, samed_masks):
#         # 计算面积
#         # 计算原始框和SAM框的面积
#         orig_area = np.sum(box_mask)
#         samed_area = np.sum(samed_mask)
#         # 计算面积比
#         area_ratio = samed_area / orig_area if orig_area > 0 else float('inf')
        
#         # 如果SAM框面积过大，使用原始框
#         if area_ratio > area_ratio_threshold:
#             final_masks.append(box_mask)
#         else:
#             final_masks.append(samed_mask)
    
#     return final_masks

In [384]:
np.save('boxes_mask.npy', boxes_mask)
np.save('samed_masks.npy', samed_masks)

In [385]:
final_masks = get_final_masks(boxes_mask, samed_masks, area_ratio_threshold=4)

In [386]:
combined_mask = np.zeros_like(final_masks[0], dtype=bool)
for i, mask in enumerate(final_masks):
    combined_mask = combined_mask | mask

    # # 同时保存单独的蒙版
    # individual_mask = (mask * 255).astype(np.uint8)
    # cv2.imwrite(f"mask_{i+1}.png", individual_mask)

# 保存组合蒙版
combined_mask = (combined_mask * 255).astype(np.uint8)
cv2.imwrite(water_mark_mask_path, combined_mask)

True

使用 combined 蒙版进行inpaint

In [388]:
from pathlib import Path

INPAINT_ANYTHING_DIR = Path(r'/root/Inpaint-Anything-main')
LAMA_CONFIG = INPAINT_ANYTHING_DIR.joinpath('lama', 'configs', 'prediction', 'default.yaml')
LAMA_CKPT = '/root/autodl-tmp/pretrained_models/big-lama'

lama_predictor, lama_predict_config = load_inpaint_model(LAMA_CONFIG, LAMA_CKPT)


In [389]:
combined_mask

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [390]:
def inpaint_img(image_array, mask):
    img_inpainted = inpaint_img_with_model(img=image_array,
                                           mask=mask,
                                           model=lama_predictor,
                                           predict_config=lama_predict_config)
    return img_inpainted

# img_path = '/root/rm_water_mark/imgs/image2_inpainted.jpg'
# image = cv2.imread(img_path)
# image_array = np.array(image)

# combined_mask_array = np.array(combined_mask)
img_clean_bg = inpaint_img(img_inpainted, combined_mask)

In [391]:
# import
import cv2
import matplotlib.pyplot as plt

cv2.imwrite(pic_clean_bg_path, img_clean_bg)
# img_inpainted_rgb = cv2.cvtColor(img_inpainted, cv2.COLOR_BGR2RGB)
# plt.imshow(img_inpainted_rgb)
# plt.axis('off')  # 不显示坐标轴
# plt.show()

True

将商品主体蒙版和无文字的干净背景融合，得到最终结果

In [392]:
# # 原图
# img_path = '/root/rm_water_mark/imgs/image2.jpg'
# # 主体蒙版
# mask_path = '/root/rm_water_mark/imgs/image2_mask.png'
# # 背景图
# bg_path = '/root/rm_water_mark/imgs/image2_bg.jpg'

# img = cv2.imread(img_path)
# mask = cv2.imread(mask_path)
# bg = cv2.imread(bg_path)
# 将img中mask以外的区域替换为bg中的内容
result = original_image.copy()
result[product_mask == 0] = img_clean_bg[product_mask == 0]

cv2.imwrite(final_img_path, result)

True

In [393]:
end_time = time.time()
print('Time cost: ', end_time - start_time)

Time cost:  15.031396627426147
